In [93]:
import os
import argparse
from pydub import AudioSegment
#trim audio and save
import numpy
import math
import scipy.io.wavfile
import librosa
import pandas
# prep piptrack
class Config(object):
    n_fft = 1024
    hop_length=220
    sr = 44000
    duration = 120
    feature = "mel"

conf = Config()    

In [94]:
len(y)
#times[i] = frames[i] * hop_length / sr

436884

In [95]:
import librosa
import numpy as np
import os
import sys
import h5py
import multiprocessing
import functools
import h5py
import threading
from pathlib import Path
from numpy import pi, convolve
from scipy.signal.filter_design import bilinear
from scipy.signal import lfilter


def extract_max(pitches, magnitudes, shape):
    new_pitches = []
    new_magnitudes = []
    for i in range(0, shape[1]):
        new_pitches.append(np.max(pitches[:,i]))
        new_magnitudes.append(np.max(magnitudes[:,i]))
    return (new_pitches,new_magnitudes)


def smooth(x, window_len=11, window='hanning'):
    if window_len<3:
        return x
    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
        raise ValueError("Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'")
    s=np.r_[2*x[0]-x[window_len-1::-1],x,2*x[-1]-x[-1:-window_len:-1]]
    if window == 'flat': #moving average
        w=np.ones(window_len,'d')
    else:
        w=eval('np.'+window+'(window_len)')
    y=np.convolve(w/w.sum(),s,mode='same')
    return y[window_len:-window_len+1]

def pitch_detect(y, sr):
    pitches, magnitudes = librosa.core.piptrack(y=y, sr=sr,hop_length=220)
    shape = pitches.shape
    nb_samples = shape[0]
    print(nb_samples)
    nb_windows = shape[1]
    print(nb_windows)
    pitches, magnitudes = extract_max(pitches, magnitudes, shape)
    pitches = smooth(pitches, window_len=5)
    return pitches




In [96]:
def get_pitch_time_series(fpath):
    y,sr=librosa.load(fpath,sr=conf.sr,res_type="kaiser_fast")
    pip=pitch_detect(y,sr)
    #change to MIDI notes
    pip=12*np.log(pip/440.0)/np.log(2.0)+69
    #section to 0.005s=5ms pitch series
    lens=pip.shape[0]
    pips = np.array([])
    for i in range(lens):
        time_pitch_pair=[]
        time_pitch_pair.append(i*220/sr)
        time_pitch_pair.append(pip[i])
        if pips.size == 0:
            pips = np.array(time_pitch_pair)
        else:
            pips = np.vstack((pips,np.array(time_pitch_pair)))
    return pips#(nums,)

In [97]:
from scipy.fftpack import fft
NFFT = 512
F_low = 5
F_high = 8

def VibratoFeature(time_pitch):
    NFFT = 512
    pitch_hop = 0.005
    freq = np.arange(0, 1 / (2 * pitch_hop), (1 / (pitch_hop * NFFT)))

    time = time_pitch[:,0]
    pitch = time_pitch[:,1]
    vib_features = np.array([])

    cur_time = 0
    time_length = 0.5 #分析长度为0.5s
    time_hop = 0.25 #滑动为0.25秒
    while cur_time<time[-1]:
        time_stamp = np.array([cur_time,cur_time+time_length])
        
        features = []
        n1 = np.argmin(np.abs(np.subtract(time, time_stamp[0])))
        n2 = np.argmin(np.abs(np.subtract(time, time_stamp[1])))
        pitch_snippet = pitch[n1:n2+1]
        time_snippet = time[n1:n2+1]

        pitch_snippet = pitch_snippet - np.mean(pitch_snippet)

        ### Likelihood
        X = np.abs(fft(pitch_snippet * np.hamming(len(pitch_snippet)), NFFT))
        Xhalf_norm = X[0:int(NFFT / 2)] / sum(X[0:int(NFFT / 2)])
        vib_region = Xhalf_norm[(freq >= F_low) & (freq <= F_high)]
        vib_power = sum(vib_region)
        vib_sum_power = sum(Xhalf_norm)
        vib_power_ratio = vib_power/vib_sum_power
        features.append(vib_power_ratio)

        ### Rate
        zero_crossings = np.where(np.diff(np.sign(pitch_snippet)))[0]
        time_zerocrossings = time_snippet[zero_crossings[0::2]]
        if len(time_zerocrossings) <=1: #不够一个周期==time_zerocrossings不到两个
            if vib_features.size == 0:
                vib_features = np.array([0.,0.,0.])
            else:
                vib_features = np.vstack((vib_features, np.array([0.,0.,0.])))
            cur_time+=time_hop
            continue
        rate = 1.0/((1.0/(len(time_zerocrossings)-1))*sum(np.abs(time_zerocrossings[0:-1]-time_zerocrossings[1:])))
        features.append(rate)

        ### Extent
        pitch_max = []
        for index in range(len(zero_crossings)-1):
            pitch_max.append(max(np.abs(pitch_snippet[zero_crossings[index]:zero_crossings[index+1]])))
        pitch_max = np.array(pitch_max)
        extent =  (0.5/(len(pitch_max)-1))*sum(np.abs(pitch_max[0:-1]+pitch_max[1:]))
        features.append(extent)

        if vib_features.size == 0:
            vib_features = np.array(features)
        else:
            vib_features = np.vstack((vib_features,np.array(features)))
        # print(features)
        cur_time+=time_hop
    return vib_features#(vib-0.25s-sections,3)-->likelihood,rate,extent

In [98]:
def vi_feat(fpath):
    return VibratoFeature(get_pitch_time_series(fpath))
    

In [99]:
fpath=r"C:\Users\gaoyu\Desktop\music try\data_resonance\all_types_F_chain.wav"
X1=vi_feat(fpath)

1025
1986


In [100]:
X1.shape

(40, 3)